<a href="https://colab.research.google.com/github/jesselynyong/RecycleRush/blob/main/Train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [175]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform
from sklearn.metrics import confusion_matrix
import seaborn as sns
from google.colab import drive
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from transformers import BeitFeatureExtractor, BeitForImageClassification
from torch.utils.data import DataLoader
import torch
import joblib

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [176]:
!pip install pipreqs


  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=be5642af25ce00ddf9ac4fa7f72543a87d2e7347b6ab36408d05924406934e2e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
!pipreqs '/content/drive/MyDrive/Hackathon/data2'

In [157]:
path_data = '/content/drive/MyDrive/Hackathon/data2'
X = []
Y = []
dico = {"paper":1, "cardboard":1, "metal":0, "trash":2, "plastic":2 }

for file_name in os.listdir(path_data):
    img_path = os.path.join(path_data, file_name)

    if os.path.isfile(img_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        image = cv2.imread(img_path)
        preprocessed_image = torch.tensor(image)
        img_name = img_path.rsplit('/', 1)[1].split('.')[0]
        img_name = ''.join(filter(str.isalpha, img_name))
        if img_name != "glass":
          Y.append(torch.tensor(dico[img_name]))
          X.append(preprocessed_image)

In [158]:
from collections import Counter
y = [h.item() for h in Y]
Counter(y)

Counter({1: 936, 2: 618, 0: 412})

In [159]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 128)

In [160]:
data_train = {"pixel_values": X_train, "labels": Y_train}
data_test = {"pixel_values": X_test, "labels": Y_test}

In [ ]:
model = BeitForImageClassification.from_pretrained("microsoft/beit-base-patch16-224-pt22k-ft22k")
feature_extractor = BeitFeatureExtractor.from_pretrained("microsoft/beit-base-patch16-224-pt22k-ft22k")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model.to(device)

In [168]:
inputs = feature_extractor(X_train, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits

In [169]:
inputs_test = feature_extractor(X_test, return_tensors="pt").to(device)
model.eval()
# Effectuer une inférence
with torch.no_grad():
    outputs_test = model(**inputs_test)
logits_test = outputs_test.logits

In [170]:
inputs = feature_extractor(X_test, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
probabilities = torch.nn.functional.softmax(logits, dim=-1)
predicted_labels = torch.argmax(probabilities, dim=1)

true_labels = torch.tensor([Y_test], dtype=torch.long)

In [171]:
import torch.nn as nn
in_features = model.classifier.in_features

classifier = nn.Sequential(nn.Linear(in_features, 3), nn.Softmax())

In [174]:
accuracy = torch.sum(predicted_labels == Y_test2) / len(Y_test2)
print(accuracy.item())

0.0


In [138]:
model.save_pretrained("/content/drive/MyDrive/Hackathon/best_model_beit3")

In [139]:
feature_extractor.save_pretrained("/content/drive/MyDrive/Hackathon/best_extractor_beit3")

['/content/drive/MyDrive/Hackathon/best_extractor_beit3/preprocessor_config.json']

In [101]:
probabilities.size()

torch.Size([394, 21841])

In [124]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BeitFeatureExtractor, BeitForImageClassification


In [125]:
logreg = LogisticRegression()

In [127]:
type(logits)

torch.Tensor

In [129]:
logreg.fit(logits.cpu().numpy(), Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [137]:
Y_pred = logreg.predict(logits_test.cpu().numpy())

# Calculer l'accuracy
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9822335025380711


In [142]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Hackathon/reg_log")

In [152]:
joblib.dump(logreg, '/content/drive/MyDrive/Hackathon/logistic_regression_model.joblib')

['/content/drive/MyDrive/Hackathon/logistic_regression_model.joblib']